<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
تشخیص نوع و منبع خرابی
</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در این قسمت کتاب‌خانه‌ها و ابزار مورد نیاز خود را 
<code>import</code>
 کنید 
 و
  فایل داده‌ها را که در پوشه‌ی 
  <code>Data</code>
ذخیره‌شده‌اند را بخوانید و وارد محیط کار خود کنید.
</font>
</p>

In [102]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [103]:
import os
import joblib
import zipfile
import itertools
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler


warnings.filterwarnings("ignore")

In [104]:
LABELS = ["fault_type", "fault_source"]

In [105]:
def load_df(source: str) -> pd.DataFrame:
    root_dir = os.path.join(os.getcwd(), source)

    # load dfs
    df_list = []
    for csv_name in os.listdir(root_dir):
        if not csv_name.endswith(".csv"):
            continue
        
        csv_path = os.path.join(root_dir, csv_name)
        df = pd.read_csv(csv_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        df = df.set_index("timestamp")
        df_list.append(df)

    # concat dfs
    flow_rate_df = [df for df in df_list if "FlowRate_L_min" in df.columns][0]
    other_df_list = [df for df in df_list if "FlowRate_L_min" not in df.columns]
    df = flow_rate_df.join(other_df_list, how="left")
    # df = df.ffill()
    df = df.fillna(0)
    
        
    return df

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
        در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
    در این قسمت شما می‌توانید بخشی از داده‌ی موجود را برای اعتبارسنجی در نظر بگیرید.
</font>
</p>

In [114]:
df = load_df("train")
df = df.dropna()

# convert the label to numerical values
fault_type_le = LabelEncoder()
fault_type_labels = fault_type_le.fit_transform(df["fault_type"])

fault_source_le = LabelEncoder()
fault_source_labels = fault_source_le.fit_transform(df["fault_source"])

df["label"] = fault_type_labels * 2 + fault_source_labels


for col in LABELS + ["label"]:
    print()
    print(df[col].value_counts() / len(df))


# Oversample each class to match the max count
counts = df["label"].value_counts()
max_count = counts.max()
df_balanced = pd.concat([
    group.sample(n=max_count, replace=True, random_state=42)
    for _, group in df.groupby("label")
])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


for col in LABELS + ["label"]:
    print()
    print(df_balanced[col].value_counts() / len(df_balanced))

    
df_balanced


fault_type
lubrication_fault    0.366667
misalignment         0.313778
bearing_fault        0.295556
sensor_fault         0.024000
Name: count, dtype: float64

fault_source
equipment_fault    0.976
sensor_fault       0.024
Name: count, dtype: float64

label
2    0.366667
4    0.313778
0    0.295556
7    0.024000
Name: count, dtype: float64

fault_type
bearing_fault        0.25
lubrication_fault    0.25
misalignment         0.25
sensor_fault         0.25
Name: count, dtype: float64

fault_source
equipment_fault    0.75
sensor_fault       0.25
Name: count, dtype: float64

label
0    0.25
2    0.25
4    0.25
7    0.25
Name: count, dtype: float64


,FlowRate_L_min,fault_type,fault_source,Pressure_kPa,pH_units,Torque_Nm,Speed_RPM,BearingTemp_C,VibVelocity_mm_s,Voltage_V,VibDisp_mm,Power_kW,Temperature_C,OilLevel_cm,VibAccel_m_s2,Humidity_pct,label
0,21.035658,bearing_fault,equipment_fault,0.000000,0.000000,84.140775,0.000000,0.000000,7.090827,0.000000,0.553224,35.996550,55.723023,5.645054,0.000000,58.459605,0
1,33.700003,bearing_fault,equipment_fault,0.000000,0.000000,50.636182,0.000000,0.000000,7.153678,0.000000,0.380592,35.006346,52.506713,7.670263,0.000000,50.245902,0
2,33.700003,lubrication_fault,equipment_fault,264.716722,0.000000,53.805668,953.013254,55.376554,6.816435,219.988258,0.378506,35.006346,51.686505,8.195285,16.828083,49.061174,2
3,30.118012,misalignment,equipment_fault,0.000000,0.000000,131.044309,0.000000,0.000000,12.319104,0.000000,0.670080,35.006346,74.263502,9.838589,0.000000,70.580470,4
4,25.933135,bearing_fault,equipment_fault,317.714328,7.433313,103.338989,1450.122015,79.013394,10.811231,224.218898,0.569553,35.006346,69.304609,9.838589,28.468883,64.716361,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,25.133508,lubrication_fault,equipment_fault,355.144965,8.124319,125.213583,1801.105479,76.588880,13.409602,234.387786,0.614185,61.519972,71.231037,13.412882,25.335934,69.213204,2
3296,21.035658,lubrication_fault,equipment_fault,245.693626,6.891546,68.606377,918.694194,58.553282,7.069412,222.889319,0.569388,36.431416,46.701021,4.569084,16.115653,66.372035,2
3297,26.908461,lubrication_fault,equipment_fault,0.000000,0.000000,124.075483,0.000000,0.000000,0.000000,0.000000,0.748692,63.885609,71.231037,11.731968,0.000000,69.077634,2
3298,21.035658,lubrication_fault,equipment_fault,0.000000,0.000000,127.794322,0.000000,0.000000,12.957081,0.000000,0.431814,38.744363,51.895862,10.771569,0.000000,62.840469,2


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>



In [115]:
X = df_balanced.drop(columns=LABELS + ["label"])
y = df_balanced.label

# normalize
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# train test spliting
X_train, X_test, y_train, y_test = train_test_split(X_scaled_res, y_res, test_size=0.01, random_state=42)

print(len(X_train), len(X_test))

model = BaggingClassifier(n_estimators=2000, random_state=42)
model.fit(X_train, y_train)

3267 33


,estimator,None
,n_estimators,2000
,max_samples,1.0
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,42
,verbose,0


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>



In [116]:
y_pred = model.predict(X_test)
print("R² Score:", r2_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Report:\n", classification_report(y_test, y_pred))

R² Score: 0.7053571428571429
Accuracy: 0.7272727272727273
Report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86         7
           2       0.44      0.57      0.50         7
           4       0.75      0.64      0.69        14
           7       1.00      1.00      1.00         5

    accuracy                           0.73        33
   macro avg       0.76      0.77      0.76        33
weighted avg       0.75      0.73      0.73        33



<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>


In [117]:
test_df = load_df("test")
test_df = test_df.fillna(0)
test_df = test_df[X.columns]

X_test_scaled = scaler.transform(test_df)
y_hat_test = model.predict(X_test_scaled)

fault_type_pred = y_hat_test // 2
fault_type_pred_cat = fault_type_le.inverse_transform(fault_type_pred)

fault_source_pred = y_hat_test % 2
fault_source_pred_cat = fault_source_le.inverse_transform(fault_source_pred)

submission = pd.DataFrame({"fault_type": fault_type_pred_cat, "fault_source": fault_source_pred_cat}).round(6)

print(len(test_df))

submission

965


,fault_type,fault_source
0,sensor_fault,sensor_fault
1,lubrication_fault,equipment_fault
2,lubrication_fault,equipment_fault
3,lubrication_fault,equipment_fault
4,lubrication_fault,equipment_fault
...,...,...
960,bearing_fault,equipment_fault
961,misalignment,equipment_fault
962,bearing_fault,equipment_fault
963,sensor_fault,sensor_fault


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [118]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'notebook.ipynb')):
    %notebook -e notebook.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['notebook.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['notebook.ipynb', 'submission.csv']
